In [5]:
from numpy import npv
import numpy
import datetime
import requests
from bs4 import BeautifulSoup
import decimal
from decimal import Decimal
import time

class DCFModel:
    def __init__(self):
        items = []
        data = self.pull_data('DGS5')[0]
        for line in data:
            item = {
                'date': line['date'],
                'values': {'value': line['value']},
                'timestamp': Decimal(str(time.mktime(
                    datetime.datetime.strptime(line['date'], "%Y-%m-%d").timetuple())))
            }
            items.append(item)
        self.risk_free = items[-1]['values']['value']


    def load_values(self):
        print("Risk Free Rate: {}".format(self.risk_free))

        query = accessor.query_dynamodb('AAPL', 2017)
        beta_mktrtn = calculateBeta_and_mktReturn('AAPL')
        print("Beta: {}".format(beta_mktrtn[0]))
        print("Market Return: {}".format(beta_mktrtn[1]))

        equity = float(query[0]['totalequity'])
        print("Total Equity: {}".format(equity))
        debt = float(query[0]['debt'])
        print("Total Debt: {}".format(debt))
        interest = float(query[0]['cashinterestpaid'])
        print("Total Interest: {}".format(interest))
        tax_rate = float(query[0]['efftaxrate'])
        print("Tax Rate: {}".format(tax_rate))
        fcf = float(query[0]['freecashflow'])
        print("Free Cash Flow: {}".format(fcf))
        shrs_outstanding = float(query[0]['weightedavebasicsharesos'])
        print("Shares Outstanding: {}".format(shrs_outstanding))
        fcf_growth = float(query[0]['fcffgrowth'])
        print("Free Cash Flow Growth: {}".format(fcf_growth))
        ebitda = float(query[0]['ebitda'])
        print("EBITDA: {}".format(ebitda))
        ev = float(query[0]['enterprisevalue'])
        print("Enterprise Value: {}".format(ev))

        ke = calculate_capm(risk_free, beta_mktrtn[0], beta_mktrtn[1])
        print("Cost of Equity: {}".format(ke))
        kd = calculate_kd(interest, debt)
        print("Cost of Debt: {}".format(kd))
        wacc = calculate_wacc(equity, debt, ke, kd, tax_rate)
        print("WACC: {}".format(wacc))
        dcf = calculateDCF(fcf, fcf_growth, wacc, shrs_outstanding, ebitda, ev)
        print("DCF Analysis Predicted Stock Price: '${:,.2f}'".format(dcf))

        return dcf


    def pull_data(self, symbol):
        url = "https://fred.stlouisfed.org/data/" + symbol + ".txt"
        hdr = {'User-Agent': 'we dont want to wait for more requests /u/m&j'}
        response = requests.get(url, stream=True, headers=hdr)
        soup = BeautifulSoup(response.text, "html.parser")
        split_txt = soup.text.splitlines()
        meta = True
        meta_dict = {}
        meta_dict['notes'] = ''
        data_dict = {}
        for el in split_txt:
            try:
                if meta:
                    if '' == el:
                        pass
                    elif ' ' == el[0]:
                        meta_dict['notes'] += el.replace('                     ', ' ')
                    elif 'DATE' in el:
                        meta = False
                    elif 'Series' in el[:10] or 'Seasonal' in el[:10] or 'Date Range' in el[:10] or 'Last' in el[:10]:
                        tab_split = el.split(None, 2)
                        tab_split = [tab_split[0] + '_' + tab_split[1], tab_split[2]]
                        meta_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
                    else:
                        tab_split = el.split(None, 1)
                        if len(tab_split) == 1:
                            meta_dict[tab_split[0].lower().replace(':', '')] = ''
                        else:
                            meta_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
                else:
                    tab_split = el.split(None, 1)
                    data_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
            except IndexError as e:
                pass

        format_data = []
        for k, v in data_dict.items():
            if v == '.':
                pass
            # format_data.append({'date': k, 'value': decimal.Decimal('NaN')})
            else:
                format_data.append({'date': k, 'value': float(v)})
        return format_data, meta_dict

    # aka cost of equity
    def calculate_capm(self, risk_free_rate, beta, market_return):
        return risk_free_rate + (beta * (market_return - risk_free_rate))

    def calculate_kd(self, tot_int, tot_debt):
        return tot_int/tot_debt

    def calculate_wacc(self, equity, debt, cost_of_equity, cost_of_debt, corporate_tax_rate):
        v = equity + debt
        return ((equity/v) * cost_of_equity) + ((debt/v) * cost_of_debt * (1 - corporate_tax_rate))

    def calculateDCF(self, current_fcf, growth_rate, WACC, shares_outstanding, ebitda, ev):
        fcf_list = []
        for x in range(0, 5):
            current_fcf = current_fcf*(1+growth_rate)
            fcf_list.append(current_fcf)
            # print(current_fcf)
        # print(fcf_list)
        # print(npv(WACC,fcf_list))
        pre_terminal_value = npv(WACC, fcf_list)
        terminal_value = ebitda * (ev/ebitda)
        return (pre_terminal_value + terminal_value)/shares_outstanding

    def calculateBeta_and_mktReturn(self, ticker):
        now = datetime.datetime.now()
        now_minus_year = datetime.datetime.now() - datetime.timedelta(days=365)
        stock = quandl.get('EOD/{}'.format(ticker), column_index='4', start_date=now_minus_year, end_date=now.date())
        market = quandl.get('EOD/SPY', column_index='4', start_date=now_minus_year, end_date=now.date())
        # market = pd.read_csv('SPY.csv')
        # print(market)
        # print(market.get('Close'))

        stock_lst = []
        mkt_lst = []
        for x in range(len(stock)):
            stock_lst.append(float(stock.get('Close')[x]))
            mkt_lst.append(float(market.get('Close')[x]))

        stock_return_list = []
        market_return_list = []
        for y in range(1, len(stock)):
            stock_return_list.append((stock_lst[y]-stock_lst[y-1])/stock_lst[y-1])
            market_return_list.append((mkt_lst[y] - mkt_lst[y - 1]) / mkt_lst[y - 1])

        # print(stock_return_list)
        # print(market_return_list)
        # print(lst)
        # stock_rtn = (stock_lst[-1] - stock_lst[0]) / (stock_lst[0])
        # print(stock_rtn)
        #
        mkt_rtn = (mkt_lst[-1] - mkt_lst[0]) / (mkt_lst[0])
        # print(mkt_rtn)

        Covariance = numpy.cov(stock_return_list, market_return_list, bias=True)[0][1]
        # print(Covariance)
        variance_mkt = numpy.var(market_return_list)
        # print(variance_mkt)
        beta = (Covariance / variance_mkt)

        return [beta, mkt_rtn]


# this test is for DHI... WACC is almost perfect based on my calculations
if __name__ == '__main__':
    dcf = DCFModel()
    dcf.load_values()

OverflowError: mktime argument out of range

In [4]:
!pip install bs4

Using legacy setup.py install for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'
